# Filtro de spam


#### [Entrenamiento:](#Entrenamiento)
- def que separe una línea en palabras y las agregue a un conjunto dado por parámetro
- def de un generator que devuelva un set de palabras por cada correo de un mbox dado por parámetro
- def que devuelve Dict de la forma {'w' : n } con w = palabra y n = Hw para 'ham.mbox' o n = Sw para 'spam.box'

#### [Clasificación:](#Clasificación)
- def que devuelva P(y|xw) para cada palabra w de un conjunto de palabras, uno por cada correo de 'news.box'
- def que devuelva lista de 15 palabras de mejor clasificación individual de un correo nuevo
- def que devuelva P(y|x1, . . . , x15) de la lista anterior
- def que devuelva lista de 0 o 1 por cada correo de 'news.mbox' en el mismo orden

#### [Incorporación:](#Incorporación)
- def que devuelva un set de palabras de un 'new-spam.mbox' o 'new-ham.mbox' que contiene un solo mensaje
- def que combine el anterior conjunto al dict de entrenamiento aumentando en 1 cada valor
- def que escriba el nuevo correo 'new-spam.mbox' o 'new-ham.mbox' en el fichero que corresponda

#### [Uso:](#Uso)
- Ejemplo de uso de los 3 procedimientos anteriores

# Entrenamiento

In [10]:
import re


# Separa al string 'line' según el regex delimiter
# Y añade los trozos al set 'words' pasado por parámetro
def process_line(line, words, delimiter = '[,. ()]|\n|\t'):
    for word in re.split(delimiter, line):
        words.add(word)


# Devuelve un set de palabras diferentes de un solo correo
def words_per_mail(mbox):    
    with open(mbox) as fp:
        lines = iter(fp.readlines())
        while True:
            words = set()
            for subject in lines:
                if subject.startswith("Subject: "):
                    break
            else:
                break

            process_line(subject[9:], words)

            for status in lines:
                if status == "X-KMail-MDN-Sent:  \n":
                    break

            intro = fromOne = False
            for line in lines:
                if line == '\n' :
                    intro = True
                    continue
                elif line.startswith('From: ') and fromOne :
                    break
                elif line.startswith('From ') and intro :
                    fromOne = True
                else:
                    intro = False
                    process_line(line, words)

            words.remove('')
            yield words


# Agrega words a dictionary, incrementando la ocurrencia en 1
def merge_to_dict(words, dictionary):
    for w in words:
        dictionary[w] = dictionary[w] + 1 if w in dictionary else 1


# Devuelve dict de palabras no repetidas y el número de correos
# en los que aparecen, del mbox dado por parámetro
def training_dict(mbox):
    dictionary = {}
    for words in words_per_mail(mbox):
        merge_to_dict(words, dictionary)
    return dictionary


In [11]:
# Devuelve dict de la forma {'word' : Sw } para spambox
# y otro dict de la forma {'word' : Hw } para hambox
def training(spambox, hambox):
    return training_dict(spambox), training_dict(hambox)


In [103]:

import mailbox

newsbox = mailbox.mbox('news.mbox')
#Mostramos las probabilidades del primer correo
for message in newsbox:
    wordsGenerator = words_per_mail(message)
    for word in wordsGenerator:
        print(word)
    break

TypeError: invalid file: <mailbox.mboxMessage object at 0x000000884575EA90>

# Clasificación

In [61]:
# Devuelve generator que recorre las palabras de cada correo de newsbox
# Devuelve un set por cada llamada, que corresponde al siguiente correo
def news_words_generator(newsbox = './news.mbox'):
    return words_per_mail(newsbox)

import mailbox

#Devuelve el número de mensajes que tiene una carpeta
def messagesPerFolder(mbox):
    i = 0
    for message in mbox:
        i+=1
    return i

#Devuelve el número de correos de una carpeta en los que aparece una palabra 
def numberMailsItAppears(word, mbox):
    count = 0
    wordsPerFolder = words_per_mail(mbox)
    for wordsPerMail in wordsPerFolder:
        for words in wordsPerMail:
            if word in words:
                count+=1
                break
    return count

#Devuelve P(y|xw) por cada palabra en un correo
def probPerWord(words):
    spambox = mailbox.mbox('spam.mbox')
    hambox = mailbox.mbox('ham.mbox')
    S = messagesPerFolder(spambox) #número de correos Spam
    H = messagesPerFolder(hambox) #número de correos no Spam
    Py = S/(S + H)
    
    Pxwy = []
    Pw = []
    Pyxw = []
    for word in words:
        Sw =  numberMailsItAppears(word, './spam.mbox') #número de correos Spam donde aparece esa palabra
        Pxwy.append(Sw/S)
        Hw = numberMailsItAppears(word, './ham.mbox') # número de correos no Spam donde aparece esa palabra
        if Sw==0 or Hw==0:
            Pw.append(0.5)
        else:
            Pw.append((Sw+Hw)/(S+H))
    #Calculamos P(y|xw) con la regla de Bayes: P(A|B) = P(B|A)*P(A)/P(B)
    counter2 = 0
    for value in Pxwy:
        Pyxw.append(value*Py/Pw[counter2])
        counter2+=1
    return Pyxw
                
                

In [96]:
from collections import namedtuple

Pair = namedtuple("Pair", ["value", "position"])

#Devuelve una lista de 15 palabras de mejor clasificación individual de un correo
def bestQualificationWords(words):
    probabilidades = probPerWord(words)
    wordsGenerator = words_per_mail(words)
    wordsList = list(wordsGenerator)
    diffZeroOne = []
    bestFifteen = []
    result = []
    #Calculamos la diferencia entre las probabilidades y 0 ó 1 (dependiendo de su valor), para ver cuáles son
    #los más próximos a éstos y escoger los 15 mejores
    counter = 0
    for prob in probabilidades: 
        if prob <= 0.5:
            diffZeroOne.append(Pair(abs(prob), counter))
        elif prob > 0.5:
            diffZeroOne.append(Pair(abs(1-prob), counter))
        counter+=1
    #Ordenamos de menor a mayor
    diffSorted = sorted(diffZeroOne)
    #Seleccionamos los 15 primeros (los de menor diferencia con 0 ó 1)
    i = 0
    while i<15:
        bestFifteen.append(Pair(diffSorted[i].value, diffSorted[i].position))
        i+=1
    #Por último, devolvemos las palabras que se corresponden a esos resultados
    for best in bestFifteen:
        result.append(wordsList[best.position])
    return result

# Incorporación

In [13]:
# Escribe el nuevo correo ubicado en la ruta 'new'
# en el fichero de ruta 'box'
def append_new_to_mbox(new, mbox):
    with open(mbox, 'a') as writable:
        with open(new) as readable:
            writable.write(readable.read())


# Actualiza dictionary a partir del set de palabras
# del único mensaje contenido en 'new_box'
def update_dict(new_box, dictionary):
    new_words = next(words_per_mail(new_box))
    merge_to_dict(new_words, dictionary)


In [14]:
# Actualiza dictionary y escribe el contenido de new_box en mbox
def incorporation(dictionary, new_box, mbox):
    update_dict(new_box, dictionary)
    append_new_to_mbox(new_box, mbox)


# Uso

In [97]:
# Archivos
newspam = './new-spam.mbox'
newham = './new-ham.mbox'
spambox = './spam.mbox'
hambox = './ham.mbox'

# Entrenamiento
spam_dict, ham_dict = training(spambox, hambox)

# Clasificación
import mailbox

newsbox = mailbox.mbox('news.mbox')
#Mostramos las probabilidades del primer correo
for message in newsbox:
    probabilidades = probPerWord(message)
    break
print(probabilidades)
#Mostramos las 15 mejores palabras del primer correo
for message in newsbox:
    palabras = bestQualificationWords(message)
    break
print(palabras)

# Incorporación a spam.box
incorporation(spam_dict, newspam, spambox)

# Incorporación a ham.box
incorporation(ham_dict, newham, hambox)

[0.33333333333333337, 0.4736842105263159, 0.16666666666666669, 0.16666666666666669, 0.5, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.6666666666666667, 0.5, 0.5, 0.33333333333333337, 0.5, 0.5, 0.5, 0.5]


TypeError: invalid file: <mailbox.mboxMessage object at 0x0000008845834470>